<a href="https://colab.research.google.com/github/sweetbaer03/KOR_Sentiment-Analysis/blob/main/NSMC_KoElectra_model(%EC%82%AC%EC%A0%84%EC%B6%94%EA%B0%80).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**NSMC Sentiment Analysis**

KoElectra 모델 사용(감정사전추가)

@misc{park2020koelectra,
  author = {Park, Jangwon},
  title = {KoELECTRA: Pretrained ELECTRA Model for Korean},
  year = {2020},
  publisher = {GitHub},
  journal = {GitHub repository},
  howpublished = {\url{https://github.com/monologg/KoELECTRA}}
}

In [1]:
!pip3 install transformers

     |████████████████████████████████| 1.5MB 5.6MB/s 
     |████████████████████████████████| 2.9MB 20.0MB/s 
     |████████████████████████████████| 890kB 44.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=c3a1ceeb3f7a72c49ad1717053b089c5909378f984c5bc1fea00e5b15378f63b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import tensorflow as tf
import torch

from transformers import ElectraTokenizer
from transformers import ElectraForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime
import urllib.request

# 2. 데이터 로드

github에서 필요한 데이터파일 다운

In [3]:
!git clone https://github.com/sweetbaer03/KOR_Sentiment-Analysis.git

Cloning into 'KOR_Sentiment-Analysis'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 21 (delta 5), reused 8 (delta 1), pack-reused 0
Unpacking objects: 100% (21/21), done.


# 3. 전처리

In [35]:
MAX_LEN = 128
batch_size = 32
epochs = 5

TOKEN_MODEL = 'monologg/koelectra-base-v3-discriminator'
TRAIN_MODEL = 'monologg/koelectra-base-v3-discriminator'

## 3.1 네이버리뷰 처리

In [5]:
# train/test set 데이터 로드
train_data = pd.read_csv("KOR_Sentiment-Analysis/data_in/ratings_train.txt", sep='\t')
test_tmp = pd.read_csv("KOR_Sentiment-Analysis/data_in/ratings_test.txt", sep='\t')

In [6]:
print(train_data.shape)
print(test_tmp.shape)

(150000, 3)
(50000, 3)


In [7]:
train_data

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


## 3.2 캐글 데이터

In [8]:
# 테스트(캐글) 데이터 로드
test_csv = pd.read_csv('KOR_Sentiment-Analysis/data_in/ko_data.csv', encoding = 'cp949')
test_csv.columns = ['id','document']
test_csv.head()

,id,document
0,0,정말 많이 울었던 영화입니다.
1,1,시간 낭비예요.
2,2,포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.
3,3,지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!
4,4,이걸 영화로 만드는 거야?얼마나 가는지 보자.


# 감정사전 처리

In [9]:
dic_data_value = pd.read_table('KOR_Sentiment-Analysis/data_in/SentiWord_Dict.txt', header=None)
dic_data_value.rename(columns={dic_data_value.columns[0] : 'document', dic_data_value.columns[1] : 'label'}, inplace=True)
dic_data_value.head()

,document,label
0,(-;,1.0
1,(;_;),-1.0
2,(^^),1.0
3,(^-^),1.0
4,(^^*,1.0


In [10]:
dic_data_value['document'] = dic_data_value['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
dic_data_value['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
dic_data_value = dic_data_value.dropna(how='any') # Null 값 제거
dic_data_value = dic_data_value.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(dic_data_value.isnull().values.any()) # Null 값이 존재하는지 확인
#감성이 없는 단어 삭제
idx_ = dic_data_value[dic_data_value['label']==0.0].index
dic_data_value = dic_data_value.drop(idx_)
print(dic_data_value.groupby(['label']).size().reset_index(name = 'count'))

False
   label  count
0   -2.0   4799
1   -1.0   5016
2    1.0   2246
3    2.0   2603


In [11]:
# 감정을 숫자로 변환
def emotion_labeling_dic(emotion):
   return{-2.0 : 0, -1.0: 0, 1.0: 1, 2.0: 1}[emotion]

emotion_labels = []

for e in dic_data_value['label']:
   emotion_labels.append(emotion_labeling_dic(e))

dic_data_value['label'] = emotion_labels
dic_data_value

,document,label
10,ㅡㅡ,0
28,ㅅ,0
41,ㄱㅅ,1
42,ㄱㅇㄷ,1
43,가격이 싸다,1
...,...,...
14849,오류,0
14851,의혹,0
14852,내팽개치다,0
14853,횡령,0


In [12]:
dic_data_value.reset_index(drop=True)

,document,label
0,ㅡㅡ,0
1,ㅅ,0
2,ㄱㅅ,1
3,ㄱㅇㄷ,1
4,가격이 싸다,1
...,...,...
14659,오류,0
14660,의혹,0
14661,내팽개치다,0
14662,횡령,0


In [13]:
print('전처리 후 사전 샘플의 개수 :',len(dic_data_value))

전처리 후 사전 샘플의 개수 : 14664


## 데이터 합치기

In [14]:
#합치기 위해 포멧 통일
train_data.drop(labels='id', axis="columns", inplace=True)

In [15]:
train = pd.concat([dic_data_value, train_data])
#인덱스 재정렬
train.reset_index(drop=True, inplace=True)
train

,document,label
0,ㅡㅡ,0
1,ㅅ,0
2,ㄱㅅ,1
3,ㄱㅇㄷ,1
4,가격이 싸다,1
...,...,...
164659,인간이 문제지.. 소는 뭔죄인가..,0
164660,평점이 너무 낮아서...,1
164661,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
164662,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


## 데이터 전처리

In [16]:
def getInputs(dataset):
  data = dataset.copy(deep=True)

  if 'document' in data.columns:
    sentences = data['document']
  else:
    sentences = data['Sentence']

  sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
  
  tokenizer = ElectraTokenizer.from_pretrained(TOKEN_MODEL, do_lower_case=False)
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  attention_masks = []
  for seq in input_ids:
      seq_mask = [float(i>0) for i in seq]
      attention_masks.append(seq_mask)

  return input_ids, attention_masks

def getIndex(dataset):
  data = dataset.copy(deep = True)
  input_index = data.index.tolist()
  return torch.tensor(input_index)


In [22]:
labels1 = train['label'].values
ratings_inputs1, ratings_masks1 = getInputs(train)

ratings_test_labels = test_tmp['label'].values
ratings_test_inputs, ratings_test_masks = getInputs(test_tmp)

test_df_inputs, test_df_masks = getInputs(test_csv)

In [23]:
print('전체 labels1 데이터의 개수: {}'.format(len(labels1)))
print('전체 ratings_inputs1 데이터의 개수: {}'.format(len(ratings_inputs1)))
print('전체 ratings_masks1 데이터의 개수: {}'.format(len(ratings_masks1)))

전체 labels1 데이터의 개수: 164664
전체 ratings_inputs1 데이터의 개수: 164664
전체 ratings_masks1 데이터의 개수: 164664


In [24]:
print('전체 ratings_test_labels 데이터의 개수: {}'.format(len(ratings_test_labels)))
print('전체 ratings_test_inputs 데이터의 개수: {}'.format(len(ratings_test_inputs)))
print('전체 ratings_test_masks 데이터의 개수: {}'.format(len(ratings_test_masks)))

전체 ratings_test_labels 데이터의 개수: 50000
전체 ratings_test_inputs 데이터의 개수: 50000
전체 ratings_test_masks 데이터의 개수: 50000


In [25]:
print('전체 test_inputs 데이터의 개수: {}'.format(len(test_df_inputs)))
print('전체 test_masks 데이터의 개수: {}'.format(len(test_df_masks)))

전체 test_inputs 데이터의 개수: 11187
전체 test_masks 데이터의 개수: 11187


In [ ]:
ratings_inputs = []
ratings_inputs.extend(ratings_inputs1)
#ratings_inputs.extend(ratings_inputs2)
#ratings_inputs.extend(ratings_inputs3)
labels = []
labels.extend(labels1)
#labels.extend(labels2)
#labels.extend(labels3)
ratings_masks = []
ratings_masks.extend(ratings_masks1)
#ratings_masks.extend(ratings_masks2)
#ratings_masks.extend(ratings_masks3)

In [ ]:
print('전체 labels 데이터의 개수: {}'.format(len(labels)))
print('전체 ratings_inputs 데이터의 개수: {}'.format(len(ratings_inputs)))
print('전체 ratings_masks 데이터의 개수: {}'.format(len(ratings_masks)))

전체 labels 데이터의 개수: 150000
전체 ratings_inputs 데이터의 개수: 150000
전체 ratings_masks 데이터의 개수: 150000


In [ ]:
ratings_inputs1

array([[   2,   44,  166, ...,    0,    0,    0],
       [   2, 7987,    5, ...,    0,    0,    0],
       [   2,    1,    3, ...,    0,    0,    0],
       ...,
       [   2, 5913, 2577, ...,    0,    0,    0],
       [   2, 6652,  494, ...,    0,    0,    0],
       [   2,  131,  494, ...,    0,    0,    0]])

마스크 씌우기

In [26]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(ratings_inputs1, labels1, random_state=2018, test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(ratings_masks1, ratings_inputs1, random_state=2018, test_size=0.1)

In [27]:
# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)

validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)			

#네이버 테스트
ratings_test_inputs = torch.tensor(ratings_test_inputs)
ratings_test_labels = torch.tensor(ratings_test_labels)
ratings_test_masks = torch.tensor(ratings_test_masks)

#결과내야할 csv
test_df_index = getIndex(test_csv)
test_df_inputs = torch.tensor(test_df_inputs)
test_df_masks = torch.tensor(test_df_masks)

In [29]:
# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

test = TensorDataset(ratings_test_inputs, ratings_test_masks, ratings_test_labels)
train_test_sampler = RandomSampler(test)
train_test_dataloader = DataLoader(test, sampler=train_test_sampler, batch_size=batch_size)

test_df_data = TensorDataset(test_df_index, test_df_inputs, test_df_masks)
test_df_sampler = RandomSampler(test_df_data)
test_df_dataloader = DataLoader(test_df_data, sampler=test_df_sampler, batch_size=batch_size)

In [30]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [36]:
# ELECTRA 모델 생성
model = ElectraForSequenceClassification.from_pretrained(TRAIN_MODEL, num_labels = 2)
model.cuda()

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [37]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 3e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 학습률을 조금씩 감소시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [38]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))


# f1-score parameter
from sklearn.metrics import f1_score
f1_score_avg = []
trues = []
preds = []

In [39]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()
  
    # 로스 초기화
    total_loss = 0

    # 스코어 초기화
    f1_score_avg = []
    trues = []
    preds = []

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.5f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        #f1
        pred_flat = np.argmax(logits, axis=1).flatten()
        trues_flat = label_ids.flatten()
        trues.extend(trues_flat)
        preds.extend(pred_flat)

        # 출력 로직과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.5f}".format(eval_accuracy/nb_eval_steps))
    print("  f1 score macro: {0: 2f}".format(f1_score(y_true=trues,y_pred= preds, labels = [0,1], average='macro')))
    print("  f1 score micro: {0: 2f}".format(f1_score(y_true=trues, y_pred= preds, labels = [0,1],average='micro')))
    print("  f1 score weighted: {0: 2f}".format(f1_score(y_true=trues, y_pred= preds, labels = [0,1],average='weighted')))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")



======== Epoch 1 / 5 ========
Training...
  Batch   500  of  4,632.    Elapsed: 0:03:26.
  Batch 1,000  of  4,632.    Elapsed: 0:06:51.
  Batch 1,500  of  4,632.    Elapsed: 0:10:16.
  Batch 2,000  of  4,632.    Elapsed: 0:13:42.
  Batch 2,500  of  4,632.    Elapsed: 0:17:07.
  Batch 3,000  of  4,632.    Elapsed: 0:20:33.
  Batch 3,500  of  4,632.    Elapsed: 0:23:59.
  Batch 4,000  of  4,632.    Elapsed: 0:27:25.
  Batch 4,500  of  4,632.    Elapsed: 0:30:51.

  Average training loss: 0.26711
  Training epcoh took: 0:31:45

Running Validation...
  Accuracy: 0.90985
  f1 score macro:  0.909813
  f1 score micro:  0.909820
  f1 score weighted:  0.909833
  Validation took: 0:01:03

======== Epoch 2 / 5 ========
Training...
  Batch   500  of  4,632.    Elapsed: 0:03:26.
  Batch 1,000  of  4,632.    Elapsed: 0:06:51.
  Batch 1,500  of  4,632.    Elapsed: 0:10:16.
  Batch 2,000  of  4,632.    Elapsed: 0:13:41.
  Batch 2,500  of  4,632.    Elapsed: 0:17:06.
  Batch 3,000  of  4,632.    Elaps

In [42]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))


# f1-score parameter
from sklearn.metrics import f1_score
f1_score_avg = []
trues = []
preds = []

In [43]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(train_test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

   #
    pred_flat = np.argmax(logits, axis=1).flatten()
    trues_flat = label_ids.flatten()
    trues.extend(trues_flat)
    preds.extend(pred_flat)
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("  f1 score macro: {0: 2f}".format(f1_score(y_true=trues,y_pred= preds, labels = [0,1], average='macro')))
print("  f1 score micro: {0: 2f}".format(f1_score(y_true=trues, y_pred= preds, labels = [0,1],average='micro')))
print("  f1 score weighted: {0: 2f}".format(f1_score(y_true=trues, y_pred= preds, labels = [0,1],average='weighted')))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of  1,563.    Elapsed: 0:00:12.
  Batch   200  of  1,563.    Elapsed: 0:00:25.
  Batch   300  of  1,563.    Elapsed: 0:00:37.
  Batch   400  of  1,563.    Elapsed: 0:00:49.
  Batch   500  of  1,563.    Elapsed: 0:01:01.
  Batch   600  of  1,563.    Elapsed: 0:01:14.
  Batch   700  of  1,563.    Elapsed: 0:01:26.
  Batch   800  of  1,563.    Elapsed: 0:01:38.
  Batch   900  of  1,563.    Elapsed: 0:01:51.
  Batch 1,000  of  1,563.    Elapsed: 0:02:03.
  Batch 1,100  of  1,563.    Elapsed: 0:02:15.
  Batch 1,200  of  1,563.    Elapsed: 0:02:27.
  Batch 1,300  of  1,563.    Elapsed: 0:02:40.
  Batch 1,400  of  1,563.    Elapsed: 0:02:52.
  Batch 1,500  of  1,563.    Elapsed: 0:03:04.

Accuracy: 0.91
  f1 score macro:  0.908987
  f1 score micro:  0.909000
  f1 score weighted:  0.908995
Test took: 0:03:12


In [47]:
tmp_test_dataloader = DataLoader(test_df_data, sampler=test_df_sampler, batch_size=1)
test_result = test_csv.copy(deep = True)
test_result['Predicted'] = 'default'
classes = [0, 1]

In [48]:
test_result

,id,document,Predicted
0,0,정말 많이 울었던 영화입니다.,default
1,1,시간 낭비예요.,default
2,2,포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.,default
3,3,지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!,default
4,4,이걸 영화로 만드는 거야?얼마나 가는지 보자.,default
...,...,...,...
11182,11182,이 영화를 커플에게 추천합니다. 영화관에 가다보면 평생 잊지 못할 추억이 하나 생길...,default
11183,11183,심심__ 그냥 한효주 cf,default
11184,11184,공감해서 눈물나는 영화. 안 보신분들이 전부 제가 울었다고 하면 의아해하실텐데 보면...,default
11185,11185,오토바이 신은 최고네요.,default


In [51]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(tmp_test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_df_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_index, b_input_ids, b_input_mask = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    idx = b_index.item()
    test_result['Predicted'][idx] = classes[np.argmax(logits)]
    

    nb_eval_steps += 1

print("")
print("Test took: {:}".format(format_time(time.time() - t0)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


  Batch   100  of    350.    Elapsed: 0:00:02.
  Batch   200  of    350.    Elapsed: 0:00:03.
  Batch   300  of    350.    Elapsed: 0:00:05.
  Batch   400  of    350.    Elapsed: 0:00:06.
  Batch   500  of    350.    Elapsed: 0:00:08.
  Batch   600  of    350.    Elapsed: 0:00:09.
  Batch   700  of    350.    Elapsed: 0:00:11.
  Batch   800  of    350.    Elapsed: 0:00:13.
  Batch   900  of    350.    Elapsed: 0:00:14.
  Batch 1,000  of    350.    Elapsed: 0:00:16.
  Batch 1,100  of    350.    Elapsed: 0:00:17.
  Batch 1,200  of    350.    Elapsed: 0:00:19.
  Batch 1,300  of    350.    Elapsed: 0:00:20.
  Batch 1,400  of    350.    Elapsed: 0:00:22.
  Batch 1,500  of    350.    Elapsed: 0:00:23.
  Batch 1,600  of    350.    Elapsed: 0:00:25.
  Batch 1,700  of    350.    Elapsed: 0:00:26.
  Batch 1,800  of    350.    Elapsed: 0:00:28.
  Batch 1,900  of    350.    Elapsed: 0:00:29.
  Batch 2,000  of    350.    Elapsed: 0:00:31.
  Batch 2,100  of    350.    Elapsed: 0:00:33.
  Batch 2,200

In [52]:
test_result

,id,document,Predicted
0,0,정말 많이 울었던 영화입니다.,1
1,1,시간 낭비예요.,0
2,2,포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.,1
3,3,지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!,1
4,4,이걸 영화로 만드는 거야?얼마나 가는지 보자.,0
...,...,...,...
11182,11182,이 영화를 커플에게 추천합니다. 영화관에 가다보면 평생 잊지 못할 추억이 하나 생길...,1
11183,11183,심심__ 그냥 한효주 cf,0
11184,11184,공감해서 눈물나는 영화. 안 보신분들이 전부 제가 울었다고 하면 의아해하실텐데 보면...,1
11185,11185,오토바이 신은 최고네요.,0


In [54]:
test_result.drop(labels='document', axis="columns", inplace=True)

In [57]:
DATA_OUT_NAME = "NSMC_KoElectra_v3_add_epo3_MaxLen:"+ str(MAX_LEN) + "_BchSize:"+ str(batch_size) + ".csv"

test_csv = test_result.to_csv(DATA_OUT_NAME, columns=['id', 'Predicted'], index=False, encoding = 'cp949')
from google.colab import files

files.download(DATA_OUT_NAME)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>